A copy from https://github.com/Heumi/BEGAN-tensorflow/tree/master/src/function

To test the difference

http://www.foldl.me/uploads/2015/conditional-gans-face-generation/paper.pdf

https://github.com/nightrome/really-awesome-gan

https://arxiv.org/pdf/1705.09966.pdf

https://arxiv.org/pdf/1703.05192.pdf

https://arxiv.org/pdf/1606.03657.pdf

https://arxiv.org/pdf/1704.02166.pdf

https://arxiv.org/pdf/1709.03842.pdf

In [ ]:
import os
import scipy.misc as scm

def make_project_dir(project_dir):
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
        os.makedirs(os.path.join(project_dir, 'models'))
        os.makedirs(os.path.join(project_dir, 'result'))
        os.makedirs(os.path.join(project_dir, 'result_test'))


def get_image(img_path):
    img = scm.imread(img_path)/255. - 0.5
    img = img[..., ::-1]  # rgb to bgr
    return img


def inverse_image(img):
    img = (img + 0.5) * 255.
    img[img > 255] = 255
    img[img < 0] = 0
    img = img[..., ::-1] # bgr to rgb
    return img


In [ ]:
import tensorflow as tf
import numpy as np


def conv2d(x, filter_shape, bias=True, stride=1, padding="SAME", name="conv2d"):
    kw, kh, nin, nout = filter_shape
    pad_size = (kw - 1) / 2

    if padding == "VALID":
        x = tf.pad(x, [[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], "SYMMETRIC")

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=filter_shape, initializer=initializer)
        x = tf.nn.conv2d(x, weight, [1, stride, stride, 1], padding=padding)

        if bias:
            b = tf.get_variable("bias", shape=filter_shape[-1], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def fc(x, output_shape, bias=True, name='fc'):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        if bias:
            b = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def pool(x, r=2, s=1):
    return tf.nn.avg_pool(x, ksize=[1, r, r, 1], strides=[1, s, s, 1], padding="SAME")


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))


def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

In [ ]:
import glob
import time
import numpy as np
import tensorflow as tf

class op_base:
    def __init__(self, sess, project_name):
        self.sess = sess

        # Train
        self.flag = True #args.flag
        self.gpu_number = 0 #args.gpu_number
        self.project = project_name #"test_began" #args.project

        # Train Data
        self.data_dir = "./Face_data/Celeba/dataset_64x64" #args.data_dir #./Data
        self.dataset = "celeba" #args.dataset  # celeba
        self.data_size = 64 #args.data_size  # 64 or 128
        self.data_opt = "crop" #args.data_opt  # raw or crop

        # Train Iteration
        self.niter = 50 #args.niter
        self.niter_snapshot = 2440 #args.nsnapshot
        self.max_to_keep = 5 #args.max_to_keep

        # Train Parameter
        self.batch_size = 16 #args.batch_size
        self.learning_rate = 1e-4 #args.learning_rate
        self.mm = 0.5 #args.momentum
        self.mm2 = 0.999 #args.momentum2
        self.lamda = 0.001 #args.lamda
        self.gamma = 0.5 #args.gamma
        self.filter_number = 64 #args.filter_number
        self.input_size = 64 #args.input_size
        self.embedding = 64 #args.embedding

        # Result Dir & File
        self.project_dir = 'assets/{0}_{1}_{2}_{3}/'.format(self.project, self.dataset, self.data_opt, self.data_size)
        self.ckpt_dir = os.path.join(self.project_dir, 'models')
        self.model_name = "{0}.model".format(self.project)
        self.ckpt_model_name = os.path.join(self.ckpt_dir, self.model_name)

        # etc.
        if not os.path.exists('assets'):
            os.makedirs('assets')
        make_project_dir(self.project_dir)

    def load(self, sess, saver, ckpt_dir):
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(ckpt_dir, ckpt_name))

In [ ]:
import glob
import time
import datetime


class Operator(op_base):
    def __init__(self, sess, project_name):
        op_base.__init__(self, sess, project_name)
        self.build_model()

    def build_model(self):
        # Input placeholder
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size, self.input_size], name='x')
        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.data_size, self.data_size, 3], name='y')
        self.kt = tf.placeholder(tf.float32, name='kt')
        self.lr = tf.placeholder(tf.float32, name='lr')

        # Generator
        self.recon_gen = self.generator(self.x)

        # Discriminator (Critic)
        d_real = self.decoder(self.encoder(self.y))
        d_fake = self.decoder(self.encoder(self.recon_gen, reuse=True), reuse=True)
        self.recon_dec = self.decoder(self.x, reuse=True)

        # Loss
        self.d_real_loss = l1_loss(self.y, d_real)
        self.d_fake_loss = l1_loss(self.recon_gen, d_fake)
        self.d_loss = self.d_real_loss - self.kt * self.d_fake_loss
        self.g_loss = self.d_fake_loss
        self.m_global = self.d_real_loss + tf.abs(self.gamma * self.d_real_loss - self.d_fake_loss)

        # Variables
        g_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "gen_")
        d_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "disc_")

        # Optimizer
        self.opt_g = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.g_loss, var_list=g_vars)
        self.opt_d = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.d_loss, var_list=d_vars)


        # initializer
        self.sess.run(tf.global_variables_initializer())

        # tf saver
        self.saver = tf.train.Saver(max_to_keep=(self.max_to_keep))

        try:
            self.load(self.sess, self.saver, self.ckpt_dir)
        except:
            # save full graph
            self.saver.save(self.sess, self.ckpt_model_name, write_meta_graph=True)

        # Summary
        if self.flag:
            tf.summary.scalar('loss/loss', self.d_loss + self.g_loss)
            tf.summary.scalar('loss/g_loss', self.g_loss)
            tf.summary.scalar('loss/d_loss', self.d_loss)
            tf.summary.scalar('loss/d_real_loss', self.d_real_loss)
            tf.summary.scalar('loss/d_fake_loss', self.d_fake_loss)
            tf.summary.scalar('misc/kt', self.kt)
            tf.summary.scalar('misc/m_global', self.m_global)
            self.merged = tf.summary.merge_all()
            self.writer = tf.summary.FileWriter(self.project_dir, self.sess.graph)

    def train(self, train_flag):
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)

        print('Shuffle ....')
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        print('Shuffle Done')

        # initial parameter
        start_time = time.time()
        kt = np.float32(0.)
        lr = np.float32(self.learning_rate)
        self.count = 0

        for epoch in range(self.niter):
            batch_idxs = len(data) // self.batch_size

            for idx in range(0, batch_idxs):
                self.count += 1

                batch_x = np.random.uniform(-1., 1., size=[self.batch_size, self.input_size])
                batch_files = data[idx * self.batch_size: (idx + 1) * self.batch_size]
                batch_data = [get_image(batch_file) for batch_file in batch_files]

                # opt & feed list (different with paper)
                g_opt = [self.opt_g, self.g_loss, self.d_real_loss, self.d_fake_loss]
                d_opt = [self.opt_d, self.d_loss, self.merged]
                feed_dict = {self.x: batch_x, self.y: batch_data, self.kt: kt, self.lr: lr}

                # run tensorflow
                _, loss_g, d_real_loss, d_fake_loss = self.sess.run(g_opt, feed_dict=feed_dict)
                _, loss_d, summary = self.sess.run(d_opt, feed_dict=feed_dict)

                # update kt, m_global
                kt = np.maximum(np.minimum(1., kt + self.lamda * (self.gamma * d_real_loss - d_fake_loss)), 0.)
                m_global = d_real_loss + np.abs(self.gamma * d_real_loss - d_fake_loss)
                loss = loss_g + loss_d

                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, "
                      "loss: %.4f, loss_g: %.4f, loss_d: %.4f, d_real: %.4f, d_fake: %.4f, kt: %.8f, M: %.8f"
                      % (epoch, idx, batch_idxs, time.time() - start_time,
                         loss, loss_g, loss_d, d_real_loss, d_fake_loss, kt, m_global))

                # write train summary
                self.writer.add_summary(summary, self.count)

                # Test during Training
                if self.count % self.niter_snapshot == (self.niter_snapshot - 1):
                    # update learning rate
                    lr *= 0.95
                    # save & test
                    self.saver.save(self.sess, self.ckpt_model_name, global_step=self.count, write_meta_graph=False)
                    self.test(train_flag)

    def test(self, train_flag=True):
        # generate output
        img_num = self.batch_size
        img_size = self.data_size

        output_f = int(np.sqrt(img_num))
        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])
        im_output_dec = np.zeros([img_size * output_f, img_size * output_f, 3])

        test_data = np.random.uniform(-1., 1., size=[img_num, self.input_size])
        output_gen = (self.sess.run(self.recon_gen, feed_dict={self.x: test_data}))  # generator output
        output_dec = (self.sess.run(self.recon_dec, feed_dict={self.x: test_data}))  # decoder output

        output_gen = [inverse_image(output_gen[i]) for i in range(img_num)]
        output_dec = [inverse_image(output_dec[i]) for i in range(img_num)]

        for i in range(output_f):
            for j in range(output_f):
                im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                    = output_gen[j + (i * output_f)]
                im_output_dec[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                    = output_dec[j + (i * output_f)]

        # output save
        if train_flag:
            scm.imsave(self.project_dir + '/result/' + str(self.count) + '_output.bmp', im_output_gen)
        else:
            now = datetime.datetime.now()
            nowDatetime = now.strftime('%Y-%m-%d_%H:%M:%S')
            scm.imsave(self.project_dir + '/result_test/gen_{}_output.bmp'.format(nowDatetime), im_output_gen)
            scm.imsave(self.project_dir + '/result_test/dec_{}_output.bmp'.format(nowDatetime), im_output_dec)

In [ ]:

class BEGAN(Operator):
    def __init__(self, sess, project_name):
        Operator.__init__(self, sess, project_name)

    def generator(self, x, reuse=None):
        with tf.variable_scope('gen_') as scope:
            if reuse:
                scope.reuse_variables()

            w = self.data_size
            f = self.filter_number
            p = "SAME"

            x = fc(x, 8 * 8 * f, name='fc')
            x = tf.reshape(x, [-1, 8, 8, f])

            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv1_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = resize_nn(x, w / 8)
                x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv2_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv2_b')
                x = tf.nn.elu(x)

            x = resize_nn(x, w / 4)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv3_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv3_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w / 2)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv4_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv4_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv5_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv5_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, 3], stride=1,  padding=p,name='conv6_a')
        return x

    def encoder(self, x, reuse=None):
        with tf.variable_scope('disc_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"

            x = conv2d(x, [3, 3, 3, f], stride=1,  padding=p,name='conv1_enc_a')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, f, 2 * f], stride=1,  padding=p,name='conv3_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 2 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 3 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = conv2d(x, [1, 1, 4 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_0')
                x = pool(x, r=2, s=2)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_b')
                x = tf.nn.elu(x)

            x = fc(x, h, name='enc_fc')
        return x

    def decoder(self, x, reuse=None):
        with tf.variable_scope('disc_') as scope:
            if reuse:
                scope.reuse_variables()

            w = self.data_size
            f = self.filter_number
            p = "SAME"

            x = fc(x, 8 * 8 * f, name='fc')
            x = tf.reshape(x, [-1, 8, 8, f])

            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = resize_nn(x, w / 8)
                x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv2_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv2_b')
                x = tf.nn.elu(x)

                x = resize_nn(x, w / 4)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv3_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv3_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w / 2)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv4_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv4_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv5_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv5_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, 3], stride=1, padding=p, name='conv6_a')
        return x

In [ ]:
import distutils.util
import os
import tensorflow as tf

''' config settings '''

project_name = "test_began"
train_flag = True

'''-----------------'''

gpu_number = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #args.gpu_number

with tf.device('/gpu:{0}'.format(gpu_number)):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        model = BEGAN(sess, project_name)

        # TRAIN / TEST
        if train_flag:
            model.train(train_flag)
        else:
            model.test(train_flag)